In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import sklearn.linear_model as sklm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
data_ml = pd.read_csv("CountryFactorData.csv", sep=",")

In [3]:

# Sort the DataFrame by the 'Date' column
data_ml = data_ml.sort_values(by='Date')
factors = ["Size", "Value", "Momentum", "Market Risk Premium"]

# Finding rows that have no returns recorded and factor data, we can predict these returns
data_to_predict = data_ml[['1M_Log_Ret', '1M_Simple_Ret']].isna().all(axis=1)
data_to_predict = data_ml[data_to_predict]
data_to_predict = data_to_predict[~data_to_predict[factors].isna().any(axis=1)]


In [4]:
# Building inputs for our Elastic_Net models

filtered_df_X = data_ml.dropna()
factors = ["Size", "Value", "Momentum", "Market Risk Premium"]
X = filtered_df_X[factors].values
Y = filtered_df_X["1M_Simple_Ret"].values
logY = filtered_df_X["1M_Log_Ret"].values

In [5]:
'''
Here is the definition of the l1_ratio parameter sourced from the library: scikit-learn
l1_ratiofloat, default=0.5
The ElasticNet mixing parameter, with 0 <= l1_ratio <= 1. 
For l1_ratio = 0 the penalty is an L2 penalty. 
For l1_ratio = 1 it is an L1 penalty. 
For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.
'''

def generate_elastic_net(alpha, Y):
    elastic_net = sklm.ElasticNet(l1_ratio=alpha)
    elastic_net.fit(X,Y)

    return elastic_net

In [6]:
# Generate Elastic Net models with different alpha values
alpha_values = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
elastic_models = [generate_elastic_net(alpha, Y) for alpha in alpha_values]

# Evaluate model performance
for alpha, elastic_model in zip(alpha_values, elastic_models):
    Y_pred = elastic_model.predict(X)
    mse = mean_squared_error(Y, Y_pred)
    r2 = r2_score(Y, Y_pred)
    print(f"Alpha={alpha}, MSE={mse:.9f}, R-squared={r2:.4f}")

Alpha=0.01, MSE=0.005523643, R-squared=0.0000
Alpha=0.1, MSE=0.005523643, R-squared=0.0000
Alpha=0.2, MSE=0.005523643, R-squared=0.0000
Alpha=0.3, MSE=0.005523643, R-squared=0.0000
Alpha=0.4, MSE=0.005523643, R-squared=0.0000
Alpha=0.5, MSE=0.005523643, R-squared=0.0000
Alpha=0.6, MSE=0.005523643, R-squared=0.0000
Alpha=0.7, MSE=0.005523643, R-squared=0.0000
Alpha=0.8, MSE=0.005523643, R-squared=0.0000
Alpha=0.9, MSE=0.005523643, R-squared=0.0000
Alpha=1.0, MSE=0.005523643, R-squared=0.0000


In [7]:
e1 = sklm.ElasticNet(l1_ratio=0.1)
e1.fit(X,Y)
e2 = sklm.ElasticNet(l1_ratio=0.9)
e2.fit(X,Y)

Y_pred1 = e1.predict(X)
mse1 = mean_squared_error(Y, Y_pred1)
Y_pred2 = e2.predict(X)
mse2 = mean_squared_error(Y, Y_pred2)

print(mse1, mse2)
print("There is no meaningful difference in the l1_ratio chosen between 0 and 1")



0.0055236428247526425 0.005523642824828948
There is no meaningful difference in the l1_ratio chosen between 0 and 1


In [8]:
# We can attempt a cross-validation grid search to find the optimal alpha and l1 parameters
'''
Here is the definition of alpha from scikit-learn:
Constant that multiplies the L1 term, controlling regularization strength. 
alpha must be a non-negative float i.e. in [0, inf).
When alpha = 0, the objective is equivalent to ordinary least squares, solved by the LinearRegression object. 
'''

e = sklm.ElasticNet()
# Define hyperparameters grid for tuning
param_grid = {'alpha': [0.1, 0.5, 1.0],
              'l1_ratio': [0.2, 0.5, 0.8]}

grid_search = GridSearchCV(estimator=e, param_grid=param_grid, cv=5)
grid_search.fit(X, Y)
best_model = grid_search.best_estimator_

print(f"Best Model: {best_model}")
print(f"The optimal alpha is {best_model.alpha} and the optimal l1_ratio is {best_model.l1_ratio}")


Best Model: ElasticNet(l1_ratio=0.8)
The optimal alpha is 1.0 and the optimal l1_ratio is 0.8


In [9]:
# Using these chosen parameters we can predict the returns and log_returns for new data.
# data_to_predict is sourced data that has no returns associated with them, but has factor data.
country_indices = data_to_predict["Ticker"]
dates = data_to_predict["Date"]

data_to_predict = data_to_predict[factors].values
final_model = sklm.ElasticNet(alpha=1.0, l1_ratio=0.8)
final_model.fit(X,Y)

final_model_log = sklm.ElasticNet(alpha=1.0, l1_ratio=0.8)
final_model_log.fit(X,logY)

Y_pred_final = final_model.predict(data_to_predict)
Y_pred_final_log = final_model_log.predict(data_to_predict)


In [10]:
print("MXEG or the Egypt Index is predicted to have the best return")

final_df = pd.DataFrame({
    "Date": dates,
    "Country Ticker": country_indices,
    "1M_Predicted_Returns": Y_pred_final,
    "1M_Predicted_LogReturns": Y_pred_final_log
})

final_df

MXEG or the Egypt Index is predicted to have the best return


,Date,Country Ticker,1M_Predicted_Returns,1M_Predicted_LogReturns
0,3/31/2024,MXAT,0.005035,0.001144
24,3/31/2024,MXIT,0.004961,0.001179
23,3/31/2024,MXIN,0.004671,0.001320
25,3/31/2024,MXJP,0.004436,0.001434
26,3/31/2024,MXKR,0.004873,0.001222
27,3/31/2024,MXMX,0.004994,0.001164
28,3/31/2024,MXMY,0.005013,0.001154
29,3/31/2024,MXNL,0.004916,0.001201
30,3/31/2024,MXNO,0.005015,0.001153
31,3/31/2024,MXNZ,0.005036,0.001143


In [11]:
# Find the row with the maximum value in column 'C'
max_row_index = final_df['1M_Predicted_Returns'].idxmax()
max_row = final_df.loc[max_row_index]

max_row


Date                       3/31/2024
Country Ticker                  MXEG
1M_Predicted_Returns         0.00504
1M_Predicted_LogReturns     0.001141
Name: 12, dtype: object